In [2]:
from wsgiref.simple_server import make_server
from wsgiref.util import request_uri

class serverson():
    def localServerApp(self, environ, start_response):
        status = "200 OK"
        headers = [(
            "Content-type", 
            "text/html; charset=utf-8"
            )]  
        
        start_response(status, headers)

        self.query_url = request_uri(environ)

        return ['teste'.encode()]

    def openAuthorization(self):
        self.redirect_uri = "http://localhost:500"
        i = len(self.redirect_uri) - self.redirect_uri[::-1].find(":")
        port = int(self.redirect_uri[i:])

        _i = self.redirect_uri.find("//") + 2
        i_ = len(self.redirect_uri[_i:]) - len(str(port)) - 1
        host = self.redirect_uri[_i:][:i_]

        server = make_server(host, port, self.localServerApp)
        # Cria o app

        # Seta a flag de reuso do endereço com false

        # Cria o server
        server = make_server("localhost", 500, self.localServerApp)

        # Inicia um try

        server.timeout = None
        server.handle_request()

        #timeout server
        #roda uma vez e pega a request

        #pega a resposta
        r = self.query_url

        print(r)


a = serverson()
a.openAuthorization()

http://localhost:500/testemano


127.0.0.1 - - [18/Apr/2025 17:40:09] "GET /testemano HTTP/1.1" 200 5


In [20]:
uri = "http://localhost:500"
i_ = len(uri) - uri[::-1].find(":")
i__ = uri[i_:].find("/")

port = (uri[i_:][:i__])

print(port)

i = uri.find("//") + 2 
_i = len(uri[i:]) - len(uri[i_:]) - 1 
host = uri[i:][:_i]

print(port) 
print(host)

50
50
localhost


In [8]:
import requests
import os, json
import webbrowser

from wsgiref.simple_server import make_server
from wsgiref.util import request_uri

OAUTH2_HEADERS = {'Content-Type' : 'application/x-www-form-urlencoded'}

OAUTH2_URL_BASE = "https://id.twitch.tv/oauth2"

oauth_authorize_params = "/authorize?response_type=code&client_id={}&redirect_uri={}&scope={}"

oauth_new_token_data = "client_id={}&client_secret={}&code={}&grant_type=authorization_code&redirect_uri={}"
oauth_refresh_token_data = "grant_type=refresh_token&refresh_token={}&client_id={}&client_secret={}"

class AuthorizationCodeGrantFlow():
    redirect_uri = ""
    url = ""
    client_id = ""
    client_secrets = ""
    query_url = ""

    def __init__(self, credentials_json: str) -> None:
        """
        Creates a :class:`AuthorizationCodeGrantFlow`.

        Args:
            credentials_json (str): The path to the credentials.json
                file that have client information.       
        """

        # Verify existence of the credentials file 
        # and if it exists, read the data, else, raise exception
        if os.path.exists(credentials_json):
            with open(credentials_json, 'r') as creds_json:
                creds_data = json.load(creds_json)

            creds_json.close()

            # Verify if got the required keys in the data.
            # Case its True save the values in the variables 
            # and construct the link to the authorization page. 
            # Case its False, raise execption
            if ("client_id" in list(creds_data) and "client_secrets" in list(creds_data) 
                and "scopes" in list(creds_data) and "redirect_uri" in list(creds_data)):

                self.client_id = creds_data["client_id"]
                self.client_secrets = creds_data["client_secrets"]
                self.scopes = creds_data["scopes"]
                self.redirect_uri = creds_data["redirect_uri"]

                # Cronstruct the scopes string                
                scopes = self.scopes[0]
                for scope in self.scopes[1:]:
                    scopes += "%20" + scope

                self.url = OAUTH2_URL_BASE + oauth_authorize_params.format(self.client_id, self.redirect_uri, scopes)

            else:
                raise Exception("Credentials file missing keys")
        else:
            raise FileNotFoundError("Credentials file not found")
        
        print(self.client_id)
        print(self.client_secrets)
        print(self.url)


auth = AuthorizationCodeGrantFlow("credentials(exemplo).json")


guydfd324shljdhdgd46ywyvzzzz278g443234
ljkfdfsdlflks9043768hfhga
https://id.twitch.tv/oauth2/authorize?response_type=code&client_id=guydfd324shljdhdgd46ywyvzzzz278g443234&redirect_uri=http://localhost:8000/oauth&scope=clips:edit%20user:write:chat
